In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [10]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [85]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2014-11-29/"
files_start = 1348
files_end = 1427


In [71]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [86]:
filenames = os.listdir(path_all)

In [87]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

['216',
 '214',
 '212',
 '210',
 '208',
 '206',
 '204',
 '202',
 '200',
 '198',
 '196',
 '194',
 '192',
 '190',
 '188',
 '186',
 '184',
 '182',
 '180',
 '178',
 '176',
 '174',
 '172',
 '170',
 '168',
 '166',
 '164',
 '162',
 '160',
 '158',
 '156',
 '154',
 '152',
 '150',
 '148',
 '146',
 '144',
 '142',
 '140',
 '138',
 '136',
 '134',
 '132',
 '130',
 '128',
 '126',
 '124',
 '122',
 '120',
 '118',
 '116',
 '114',
 '112',
 '110',
 '108',
 '106',
 '104']

In [88]:
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2014-11-30 10:06:00,1,blank sol,0.001136,0.000262,0.001029,0.000784,0.005507,0.001401,0.003911,0.001217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30 10:06:00,2,blank sol,0.001137,0.000266,0.000962,0.000695,0.005093,0.001264,0.003595,0.001160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30 10:06:00,3,blank sol,0.001144,0.000268,0.000913,0.000716,0.004948,0.001228,0.003495,0.001082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30 10:06:00,4,blank sol,0.001134,0.000268,0.000891,0.000655,0.004694,0.001167,0.003343,0.001044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30 10:06:00,5,blank sol,0.001160,0.000257,0.000793,0.000588,0.004281,0.001047,0.002990,0.000966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30 10:06:00,6,blank sol,0.001166,0.000240,0.000765,0.000606,0.004121,0.001031,0.002894,0.000932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30 10:06:00,7,blank sol,0.001174,0.000281,0.000800,0.000582,0.004271,0.001195,0.003009,0.000970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30 10:06:00,8,blank sol,0.001123,0.000258,0.000834,0.000650,0.004518,0.001149,0.003202,0.000991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-11-30 10:06:00,9,blank sol,0.001156,0.000290,0.000760,0.000581,0.004171,0.001057,0.002944,0.000914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[df_all["cycle"] == 1]["sample"]
df_all_mean

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2),sample
date,,,,,,,,,,,,,,,,,,,,,
2014-11-29 19:17:00,0.001393,0.000463,0.000682,0.000467,0.003301,0.000805,0.002265,0.000727,0.001537,0.000050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2014-11-29 19:33:00,0.001414,0.000474,0.000707,0.000495,0.003473,0.000853,0.002398,0.000770,0.001607,0.000050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2014-11-29 19:49:00,0.001351,0.000433,0.000717,0.000503,0.003521,0.000859,0.002433,0.000775,0.001623,0.000052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2014-11-29 20:04:00,0.001372,0.000497,0.000812,0.000587,0.004011,0.001005,0.002790,0.000883,0.001930,0.000094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2014-11-29 20:20:00,0.000796,0.000289,0.000404,0.000282,0.002227,0.000479,0.001866,0.000564,0.001083,0.000039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2014-11-29 20:36:00,0.001371,0.000393,0.000694,0.000495,0.003474,0.000860,0.002419,0.000777,0.001597,0.000052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2014-11-29 20:52:00,0.001313,0.000413,0.000845,0.000619,0.004237,0.001056,0.002963,0.000936,0.001931,0.000056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2014-11-29 21:07:00,0.001322,0.000406,0.000885,0.000639,0.004453,0.001116,0.003103,0.000987,0.002042,0.000074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2014-11-29 21:23:00,0.001295,0.000409,0.000697,0.000504,0.003537,0.000872,0.002443,0.000774,0.001659,0.000067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol


In [91]:
df_all_mean.to_csv(path_sav + "blanks_mean.csv", mode='a', header=False)

In [215]:
df_internal_sub.to_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm.csv", mode='a', header=False)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")